## Library Installations

The code below installs the necessary Python libraries required for building and running a question-and-answer agent using LangChain, IBM Watsonx AI, and other supporting tools. Each library is installed using the pip package manager. The | tail -n 1 part ensures only the last line of the installation output is displayed, keeping the output concise.

In [1]:
!pip install langchain==0.2.6 | tail -n 1
!pip install langchain-community==0.2.6 | tail -n 1
!pip install ibm-watsonx-ai==1.0.10 | tail -n 1
!pip install langchain_ibm==0.1.8 | tail -n 1
!pip install wget==3.2 | tail -n 1
!pip install sentence-transformers==3.0.1 | tail -n 1
!pip install chromadb==0.5.3 | tail -n 1
!pip install pydantic==2.8.0 | tail -n 1
!pip install sqlalchemy==2.0.30 | tail -n 1

## Initializing IBM Watsonx AI Client

This code sets up the IBM Watsonx AI client by initializing the necessary credentials and creating a client instance. This client will be used to interact with IBM's machine learning services.

To learn more on how to get your credentials from IBM Cloud, follow the instructions in the link below under `Creating an API key in the console`
https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui
.You might have to create an IBM Cloud Account, if you haven't already.

Or follow the guided project from IBMSKillsBuild Here to run the project lab in an already provided environment. - https://cognitiveclass.ai/courses/build-a-grounded-q-a-agent-with-langchain-granite-and-rag

In [2]:
# Import necessary modules from the ibm_watsonx_ai library
from ibm_watsonx_ai import APIClient
from ibm_watsonx_ai import Credentials
import os

# Initialize the credentials with the specified URL for IBM Watsonx AI service
credentials = Credentials(
    url = "https://us-south.ml.cloud.ibm.com",  # IBM Watsonx AI service URL
    # The api_key parameter would typically be added here to authenticate the client
    # api_key = os.getenv('IBM_WATSON_API_KEY')  # Example of fetching API key from environment variables
)

# Create an API client using the initialized credentials
client = APIClient(credentials)

# Set the project ID for the specific IBM Watsonx project
project_id = "skills-network"  # Project identifier

## Loading and Splitting Text Documents

This code downloads a text file if it does not exist locally, loads the document using LangChain's TextLoader, and then splits the document into chunks using CharacterTextSplitter. This is useful for processing large text documents in smaller, manageable pieces.

In [3]:
# Import necessary modules
import requests  # Import the requests module to handle HTTP requests
from langchain.document_loaders import TextLoader  # Import TextLoader from LangChain to load text documents
from langchain.text_splitter import CharacterTextSplitter  # Import CharacterTextSplitter to split text into chunks

# Define filename and URL
filename = 'text.txt'  # Name of the file to be downloaded and saved locally
url = 'Independent Labs/Build a grounded Q/text.txt'  # URL of the file to be downloaded. Provide your own url according to the file location path

# Download the file if it does not exist
if not os.path.isfile(filename):  # Check if the file does not exist locally
    response = requests.get(url)  # Send a GET request to the specified URL
    with open(filename, 'wb') as f:  # Open the file in write-binary mode
        f.write(response.content)  # Write the content of the response to the file

# Load the document
loader = TextLoader(filename)  # Initialize the TextLoader with the filename
documents = loader.load()  # Load the document into a variable

# Split the document into chunks using CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)  # Initialize CharacterTextSplitter with chunk size of 1000 characters and no overlap
texts = text_splitter.split_documents(documents)  # Split the loaded documents into chunks

# Print the number of chunks created
print(f"Number of chunks: {len(texts)}")  # Output the number of chunks created

Number of chunks: 2


### Creating and Storing Embeddings with Watsonx and Chroma

Now let's set up an embedding model using IBM Watsonx AI, embeds documents using this model, and stores these embeddings in a Chroma vector store. It also prints sample embedding vectors for a subset of the documents.

In [4]:
# Import necessary modules
from ibm_watsonx_ai.foundation_models.utils import get_embedding_model_specs  # Import utility to get embedding model specifications
from langchain_ibm import WatsonxEmbeddings  # Import WatsonxEmbeddings class from langchain_ibm
from ibm_watsonx_ai.foundation_models.utils.enums import EmbeddingTypes  # Import enumeration of embedding types
from langchain.vectorstores import Chroma  # Import Chroma vector store from LangChain

# Retrieve and print embedding model specifications
get_embedding_model_specs(credentials.get('url'))  # Retrieve and print embedding model specifications from the IBM Watsonx AI service

# Part 1: Create Embedding Model
# Set up the WatsonxEmbeddings object
embeddings = WatsonxEmbeddings(
    model_id=EmbeddingTypes.IBM_SLATE_30M_ENG.value,  # Specify the model ID for the embedding model
    url=credentials["url"],  # URL for the IBM Watsonx AI service
    project_id=project_id  # Project ID for the specific IBM Watsonx project
)

# Part 2: Embed Documents and Store
# Embed the documents and store the embeddings in Chroma vector store
docsearch = Chroma.from_documents(texts, embeddings)  # Embed the documents and store the embeddings in Chroma vector store

# Let us print several embedding vectors.
# Generate and print embedding vectors for a sample of the documents
sample_texts = texts[:3]  # Taking a sample of 3 documents for demonstration
sample_embeddings = embeddings.embed_documents([doc.page_content for doc in sample_texts])  # Generate embeddings for the sample documents

# Print the sample embedding vectors
print("Sample Embedding Vectors:")
for i, embedding in enumerate(sample_embeddings):  # Iterate over the sample embeddings
    print(f"Document {i + 1} Embedding Vector: Length: {len(embedding)}; {embedding}")  # Print the length and the embedding vector

Sample Embedding Vectors:
Document 1 Embedding Vector: Length: 384; [0.014287684, -0.018624494, 0.10425287, 0.009707264, 0.044396505, 0.034498498, -0.0011637486, -0.019179942, 0.045551687, 0.009745523, -0.008944669, -0.009076451, 0.0029893608, 0.020455262, -0.051753055, 0.15330191, 0.02954992, 0.0012786002, 0.012898027, -0.019495677, -0.010997851, -0.104380496, 0.024528699, 0.0018810942, 0.04237788, 0.085882865, 0.03287752, 0.00547694, 0.03549624, 0.07000522, 0.02362027, 0.008426686, 0.082991414, 0.0055107134, 0.08932523, 0.0149316825, 0.07735054, 0.03059676, 0.052171387, -5.2096216e-06, 0.0052892324, 0.010849655, 0.001597183, -0.0033374417, 0.11880174, 0.05441545, 0.039025236, -0.020059828, 0.04031473, 0.00061696453, 0.036925007, 0.097642355, 0.05062332, 0.052106366, 0.023463836, 0.0132824015, 0.050097454, -0.0021754769, 0.025557738, 0.02594622, 0.016432146, 0.122749515, 0.02371394, -0.025632314, 0.043104425, -0.016782181, -0.047486678, 0.0045230095, 0.0420029, 0.04656813, 0.0301889, 

In [5]:
# help(WatsonxEmbeddings)

## Setting Up the Model ID for IBM Watsonx AI

This code sets the model_id to specify which model type to use from the ModelTypes enumeration provided by IBM Watsonx AI. In this case, it selects the GRANITE_13B_CHAT_V2 model, which is suited for conversational AI tasks.

In [6]:
# Import the ModelTypes enumeration from the ibm_watsonx_ai library
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes

# Set the model_id to a specific model type from the ModelTypes enumeration
model_id = ModelTypes.GRANITE_13B_CHAT_V2  # Selects the GRANITE_13B_CHAT_V2 model for chat applications

### Configuring Parameters for Text Generation with IBM Watsonx AI

This code sets up a dictionary of parameters used for configuring text generation tasks in IBM Watsonx AI. It specifies the decoding method, token limits, and stop sequences to control how the model generates text.

In [7]:
# Import necessary modules for configuring text generation parameters
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams  # Import parameter names for text generation
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods  # Import available decoding methods

# Define parameters for text generation
parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,  # Set decoding method to Greedy, which selects the most likely next token
    GenParams.MIN_NEW_TOKENS: 1,  # Minimum number of new tokens to generate
    GenParams.MAX_NEW_TOKENS: 100,  # Maximum number of new tokens to generate
    GenParams.STOP_SEQUENCES: ["\n"],  # Stop generating tokens when encountering a newline character
}

## Setting Up IBM Watsonx LLM with Configuration Parameters

This code initializes a WatsonxLLM object from the langchain_ibm library using specific configuration parameters. This setup is used to interact with the IBM Watsonx AI model, specifically the GRANITE_13B_CHAT_V2 model, and apply the defined text generation parameters.

In [8]:
# Import the WatsonxLLM class from the langchain_ibm library
from langchain_ibm import WatsonxLLM  # Import the class that allows interaction with IBM Watsonx LLM

# Initialize the WatsonxLLM object with the specified configuration
watsonx_granite = WatsonxLLM(
    model_id=model_id.value,  # Set the model ID to GRANITE_13B_CHAT_V2, which is a specific pre-trained model
    url=credentials.get("url"),  # Provide the URL for the IBM Watsonx AI service endpoint
    project_id=project_id,  # Specify the project ID to use with the model
    params=parameters  # Pass the text generation parameters defined earlier (e.g., decoding method, token limits)
)

## Creating a Retrieval-Based Question-Answering Chain

This code initializes a RetrievalQA chain from the langchain library. It sets up a question-answering system that uses the watsonx_granite language model and a document retriever to answer questions based on a retrieval-augmented approach.

In [9]:
# Import the RetrievalQA class from langchain.chains
from langchain.chains import RetrievalQA

# Initialize the RetrievalQA chain
qa = RetrievalQA.from_chain_type(
    llm=watsonx_granite,  # The WatsonxLLM instance to use for generating answers
    chain_type="stuff",  # Specifies the type of chain; "stuff" indicates a basic retrieval-based QA setup
    retriever=docsearch.as_retriever()  # Document retriever to fetch relevant documents for answering questions
)

## Performing a Query with the Retrieval-Based QA System

This code snippet demonstrates how to use the RetrievalQA chain to answer a specific query. The query is passed to the qa object (the initialized RetrievalQA chain), which utilizes the configured document retriever and language model to generate an answer.

Explanation:
- Importing RetrievalQA:

RetrievalQA is a class in the langchain library that combines document retrieval with a question-answering model to answer questions based on retrieved documents.
Initializing RetrievalQA:

- llm: The large language model instance (watsonx_granite) used to generate responses to queries.
- chain_type: Specifies the type of QA chain to use. "stuff" indicates a basic retrieval-based QA system where documents are retrieved and then used to generate answers.
- retriever: The document retriever (docsearch.as_retriever()) that fetches relevant documents based on the input query, which are then used by the LLM to generate answers.


This setup creates a question-answering system that leverages both document retrieval and a language model to provide accurate answers to user queries.

In [10]:
# Define a query to be answered
query = "What did Emma share with the children during their visit?"

# Use the RetrievalQA chain to get an answer for the query
result = qa.invoke(query)  # Invoke the QA system with the provided query

# Print the result of the query
print(result) 

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


{'query': 'What did Emma share with the children during their visit?', 'result': ' Emma shared stories about her experiences gardening with her grandmother.\n'}


## Let's try other examples

In [11]:
query = "How did the community garden contribute to the sense of camaraderie among the members?"
qa.invoke(query)

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


{'query': 'How did the community garden contribute to the sense of camaraderie among the members?',
 'result': " The community garden contributed to the sense of camaraderie among the members by providing a shared space where people could come together and work on a common goal. By working together in the garden, members were able to build relationships and foster a sense of community. Additionally, the garden served as a venue for social gatherings, such as the picnic at the end of the field trip, where members could enjoy each other's company and share their harvests. This common space and shared activities helped to create a"}

In [12]:
query = "Why did the local school visit the community garden?"
qa.invoke(query)

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


{'query': 'Why did the local school visit the community garden?',
 'result': ' The local school visited the community garden for a field trip to learn about how plants grow and the importance of taking care of the environment.'}